**«Pusher»** — это многосуставная роботизированная рука, очень похожая на человеческую. Её цель — переместить целевой цилиндр (называемый объектом ) в заданное положение с помощью конечного органа робота (называемого кончиком пальца ). Робот состоит из плечевого, локтевого, предплечьевого и лучезапястного суставов.

Действие представляет собой крутящие моменты, приложенные к шарнирным соединениям.Box(-2, 2, (7,), float32)(a, b)

**Пространство наблюдения состоит из следующих частей (по порядку):**

qpos (7 элементов): Значения положения частей тела робота.

qvel (7 элементов): Скорости этих отдельных частей тела (их производных).

xpos (3 элемента): Координаты кончика пальца толкателя.

xpos (3 элемента): Координаты перемещаемого объекта.

xpos (3 элемента): Координаты позиции цели.

Пространство наблюдения представляет собой пространство, в котором имеются следующие элементы:Box(-Inf, Inf, (17,), float64)

**Общая награда составляет:** reward = reward_dist + reward_ctrl + reward_near.


Подробнее https://gymnasium.farama.org/environments/mujoco/pusher/

In [1]:
import gymnasium as gym

# Выводим все доступные среды
for env_id in gym.envs.registry:
    print(env_id)


CartPole-v0
CartPole-v1
MountainCar-v0
MountainCarContinuous-v0
Pendulum-v1
Acrobot-v1
phys2d/CartPole-v0
phys2d/CartPole-v1
phys2d/Pendulum-v0
LunarLander-v3
LunarLanderContinuous-v3
BipedalWalker-v3
BipedalWalkerHardcore-v3
CarRacing-v3
Blackjack-v1
FrozenLake-v1
FrozenLake8x8-v1
CliffWalking-v1
CliffWalkingSlippery-v1
Taxi-v3
tabular/Blackjack-v0
tabular/CliffWalking-v0
Reacher-v2
Reacher-v4
Reacher-v5
Pusher-v2
Pusher-v4
Pusher-v5
InvertedPendulum-v2
InvertedPendulum-v4
InvertedPendulum-v5
InvertedDoublePendulum-v2
InvertedDoublePendulum-v4
InvertedDoublePendulum-v5
HalfCheetah-v2
HalfCheetah-v3
HalfCheetah-v4
HalfCheetah-v5
Hopper-v2
Hopper-v3
Hopper-v4
Hopper-v5
Swimmer-v2
Swimmer-v3
Swimmer-v4
Swimmer-v5
Walker2d-v2
Walker2d-v3
Walker2d-v4
Walker2d-v5
Ant-v2
Ant-v3
Ant-v4
Ant-v5
Humanoid-v2
Humanoid-v3
Humanoid-v4
Humanoid-v5
HumanoidStandup-v2
HumanoidStandup-v4
HumanoidStandup-v5
GymV21Environment-v0
GymV26Environment-v0


In [2]:
# ==============================
# 1. Установка зависимостей
# ==============================
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install gymnasium[mujoco] mujoco pyvirtualdisplay tqdm matplotlib stable-baselines3 > /dev/null

# ==============================
# 2. Импорты
# ==============================
import gymnasium as gym
import numpy as np
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.monitor import Monitor
from pyvirtualdisplay import Display
import base64, io, glob
from gymnasium.wrappers import RecordVideo
from IPython.display import HTML, display as ipy_display

# ==============================
# 3. Виртуальный дисплей (для Colab)
# ==============================
display = Display(visible=0, size=(1400, 900))
display.start()

# ==============================
# 4. Вспомогательные функции для видео
# ==============================
def wrap_env(env, folder="./video", name_prefix="pusher"):
    env = RecordVideo(env, video_folder=folder, name_prefix=name_prefix,
                      episode_trigger=lambda x: True)
    return env

def show_video(video_folder="./video"):
    mp4list = glob.glob(f'{video_folder}/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[-1]
        video = io.open(mp4, 'rb').read()
        encoded = base64.b64encode(video)
        return HTML(f"""
            <video width=800 controls>
                <source src="data:video/mp4;base64,{encoded.decode('utf-8')}" type="video/mp4" />
            </video>
        """)
    else:
        return "Видео не найдено"

# ==============================
# 5. Создание векторной среды
# ==============================
def make_env():
    def _init():
        env = gym.make("Pusher-v5")
        env = Monitor(env)  # логируем метрики
        return env
    return _init

# создаем 8 параллельных сред
n_envs = 8
vec_env = SubprocVecEnv([make_env() for _ in range(n_envs)])

# ==============================
# 6. Обучение SAC
# ==============================
# policy_kwargs можно настроить (глубина сети)
model = SAC(
    "MlpPolicy",
    vec_env,
    verbose=1,
    batch_size=256,
    learning_rate=3e-4,
    train_freq=1,
    gradient_steps=1,
    buffer_size=1000000,
    tensorboard_log="./sac_pusher_log/"
)

# учим 1e6 шагов (~1 час на CPU, быстрее на GPU)
model.learn(total_timesteps=1_000_000)

# ==============================
# 7. Тестирование и запись видео
# ==============================
env_test = gym.make("Pusher-v5", render_mode="rgb_array")
env_test = wrap_env(env_test, folder="./video", name_prefix="pusher_sac")

obs, info = env_test.reset()
done, truncated = False, False
ep_reward = 0

while not (done or truncated):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = env_test.step(action)
    ep_reward += reward

env_test.close()
print("Episode reward:", ep_reward)

# ==============================
# 8. Показ видео
# ==============================
ipy_display(show_video("./video"))


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Using cpu device
Logging to ./sac_pusher_log/SAC_1


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Выходные данные были обрезаны до нескольких последних строк (5000).
|    episodes        | 8520     |
|    fps             | 199      |
|    time_elapsed    | 4273     |
|    total_timesteps | 852000   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 100      |
|    ep_rew_mean     | -35.8    |
| time/              |          |
|    episodes        | 8524     |
|    fps             | 199      |
|    time_elapsed    | 4277     |
|    total_timesteps | 852800   |
| train/             |          |
|    actor_loss      | 36.2     |
|    critic_loss     | 0.0301   |
|    ent_coef        | 0.0089   |
|    ent_coef_loss   | -0.124   |
|    learning_rate   | 0.0003   |
|    n_updates       | 106587   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 100      |
|    ep_rew_mean     | -35.8    |
| time/              |          |
|    episodes 

/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"


Episode reward: -37.02287248868887


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
